# Pythonの性能とCython

Python でアプリケーションを作るうえでの懸念事項の一つとして **パフォーマンス** がある。パフォーマンス問題は時間(CPU処理時間)と空間(メモリー使用量)の両方にわたる。ここでは、Pythonスクリプトのパフォーマンスの阻害要因と Cython を用いた改善方法について述べる。パフォーマンスの問題は、ボトルネックを特定したあとアルゴリズムの選択が正しいかどうかの見極めが第一歩であるが、ここでは、それ以降の問題、正しアルゴリズムを効果的に実装するための改善にフォーカスする。

## より効率的な記述方法

Cython に話しを進める前に、まずは pure Python で少しでも効率的になる書き方とその効果を検証してみる。これは、Python の特性を生かしたコーディングであるが、Pythonの実装のうち CPython と呼ばれる標準実装に固有であるケースもあることには留意すること。

以下では、関数のバイトコードを見ることがあるので、標準モジュール dis の関数 dis をインポートしておく。

In [1]:
from dis import dis

### リスト生成方法の比較

Python で list は頻繁に使用されるツールの代表である。この list の生成コストを、3つの方法で比べる。
- ls_append: list オブジェクトの append メソッドで一要素ずつ追加。
- ls_generator: ジェネレータ関数を定義して、list 関数にジェネレータオブジェクトを渡す。
- ls_inner: リスト内包表記を使う。

In [2]:
def ls_append(n):
    s = []
    for i in xrange(n):
        s.append(i)
        
def ls_generator(n):
    def nup(n):
        for i in xrange(n):
            yield i
    list(nup(n))
    
def ls_inner(n):
    [i for i in xrange(n)]

こらの関数を %timeit マジックコマンドで処理時間を測定する。

In [3]:
for f in [ls_append, ls_generator, ls_inner]:
    %timeit f(10000)

1000 loops, best of 3: 719 µs per loop
1000 loops, best of 3: 443 µs per loop
1000 loops, best of 3: 256 µs per loop


リファレンスなどでも推奨されているようにリスト内包表記が際立っていることがわかる。この違いがどこにあるのかバイトコードから追ってみる。

まずは ls_append:

In [4]:
dis(ls_append)

  2           0 BUILD_LIST               0
              3 STORE_FAST               1 (s)

  3           6 SETUP_LOOP              33 (to 42)
              9 LOAD_GLOBAL              0 (xrange)
             12 LOAD_FAST                0 (n)
             15 CALL_FUNCTION            1
             18 GET_ITER            
        >>   19 FOR_ITER                19 (to 41)
             22 STORE_FAST               2 (i)

  4          25 LOAD_FAST                1 (s)
             28 LOAD_ATTR                1 (append)
             31 LOAD_FAST                2 (i)
             34 CALL_FUNCTION            1
             37 POP_TOP             
             38 JUMP_ABSOLUTE           19
        >>   41 POP_BLOCK           
        >>   42 LOAD_CONST               0 (None)
             45 RETURN_VALUE        


19 FOR_ITER から 38 JUMP_ABSOLUTE までがループの本体である。このループ本体内で毎度 28 LOAD_ATTR によるメソッドの検索が行なわれているが、これは間違いなくオーバーヘッドである。そこで、この点を改善した ls_append2 を定義してみた。

In [5]:
def ls_append2(n):
    s = []
    a = s.append
    for i in xrange(n):
        a(i)

In [6]:
for f in [ls_append2]:
    %timeit f(10000)

1000 loops, best of 3: 396 µs per loop


実測としても ls_append よりも改善した事がはっきりわかる。この事はメソッド探索のオーバヘッドの大きさを示している。しかし、内包表記には及ばない。今度は ls_append2 と ls_inner のバイトコードを比較してみる。

In [7]:
dis(ls_append2)

  2           0 BUILD_LIST               0
              3 STORE_FAST               1 (s)

  3           6 LOAD_FAST                1 (s)
              9 LOAD_ATTR                0 (append)
             12 STORE_FAST               2 (a)

  4          15 SETUP_LOOP              30 (to 48)
             18 LOAD_GLOBAL              1 (xrange)
             21 LOAD_FAST                0 (n)
             24 CALL_FUNCTION            1
             27 GET_ITER            
        >>   28 FOR_ITER                16 (to 47)
             31 STORE_FAST               3 (i)

  5          34 LOAD_FAST                2 (a)
             37 LOAD_FAST                3 (i)
             40 CALL_FUNCTION            1
             43 POP_TOP             
             44 JUMP_ABSOLUTE           28
        >>   47 POP_BLOCK           
        >>   48 LOAD_CONST               0 (None)
             51 RETURN_VALUE        


In [8]:
dis(ls_inner)

 13           0 BUILD_LIST               0
              3 LOAD_GLOBAL              0 (xrange)
              6 LOAD_FAST                0 (n)
              9 CALL_FUNCTION            1
             12 GET_ITER            
        >>   13 FOR_ITER                12 (to 28)
             16 STORE_FAST               1 (i)
             19 LOAD_FAST                1 (i)
             22 LIST_APPEND              2
             25 JUMP_ABSOLUTE           13
        >>   28 POP_TOP             
             29 LOAD_CONST               0 (None)
             32 RETURN_VALUE        


異なるのは、ls_append2 の

    34 LOAD_FAST                2 (a)
    37 LOAD_FAST                3 (i)
    40 CALL_FUNCTION            1
    43 POP_TOP             
             
と ls_inner の

    19 LOAD_FAST                1 (i)
    22 LIST_APPEND              2

の箇所である。ls_append2 は一般的な CALL_FUNCTION 命令に対して、ls_inner は LIST_APPEND という list オブジェクトに特化した命令である。前者の CALL_FUNCTION 命令ではメソッドオブジェクトから self 引数のとりだし、引数チェックなどが行なわれるが、後者は直ちに PyList_Append を呼び出す。この汎用的動作と特殊化した動作との違いがパフォーマンスとなって現われる。

### 組み込み名、グローバル変数、ローカル変数の参照コスト

ls_append2 の例では、ローカル変数にメソッドオブジェクト(への参照)をキャッシュすることで属性探索のコストを回避できることを見た。ここでは、組み込みの名前やグローバル変数、ローカル変数からオブジェクトへアクセスする際のコストの違いを調べる。

組み込み関数の type へのリファレンスを取得する3つの関数を `t0`, `t1`, `t2` を定義してその処理時間を計測する。それぞれの関数の特徴は以下のようになっている。

- t0: 組み込み名の `type` のままアクセス。
- t1: 一度モジュールのグローバル変数 `g_type` に type への参照を保持して `g_type` によるアクセス。
- t2: 関数のローカル変数 `t` に type への参照を保持して `t` によるアクセス。

In [9]:
g_type = type
def t0(n):
    for _ in xrange(n):
        type              # builtin
def t1(n):
    for _ in xrange(n):
        g_type            # module global
def t2(n):
    t = type
    for _ in xrange(n):
        t                 # local
def t3(n):
    for _ in xrange(n):
        pass             # do nothing

n = 1000000
for t in [t0, t1, t2, t3]:
    %timeit t(n)

10 loops, best of 3: 30 ms per loop
10 loops, best of 3: 26.6 ms per loop
10 loops, best of 3: 19.9 ms per loop
100 loops, best of 3: 13.3 ms per loop


この結果から、t0, t1, t2 での名前解決のコストはそれぞれ 19ms, 14ms, 7ms とわかった。まず、t0 と t1 の違いであるが、以下のようにバイトコードは同一である。

In [10]:
dis(t0)

  3           0 SETUP_LOOP              24 (to 27)
              3 LOAD_GLOBAL              0 (xrange)
              6 LOAD_FAST                0 (n)
              9 CALL_FUNCTION            1
             12 GET_ITER            
        >>   13 FOR_ITER                10 (to 26)
             16 STORE_FAST               1 (_)

  4          19 LOAD_GLOBAL              1 (type)
             22 POP_TOP             
             23 JUMP_ABSOLUTE           13
        >>   26 POP_BLOCK           
        >>   27 LOAD_CONST               0 (None)
             30 RETURN_VALUE        


In [11]:
dis(t1)

  6           0 SETUP_LOOP              24 (to 27)
              3 LOAD_GLOBAL              0 (xrange)
              6 LOAD_FAST                0 (n)
              9 CALL_FUNCTION            1
             12 GET_ITER            
        >>   13 FOR_ITER                10 (to 26)
             16 STORE_FAST               1 (_)

  7          19 LOAD_GLOBAL              1 (g_type)
             22 POP_TOP             
             23 JUMP_ABSOLUTE           13
        >>   26 POP_BLOCK           
        >>   27 LOAD_CONST               0 (None)
             30 RETURN_VALUE        


どちらも 19 LOAD_GLOBAL によって名前の解決を行っている。実行時のこれらの違いは LOAD_GLOBAL の動作による。LOAD_GLOBAL はグローバル辞書を検索して見つからなかった場合に組み込みの辞書を検索する。t0 では二つの辞書を検索しているため遅くなっているのである。

次に t2 のバイトコードであるが、LOAD_GLOBAL ではなく 25 LOAD_FAST になっている。

In [12]:
dis(t2)

  9           0 LOAD_GLOBAL              0 (type)
              3 STORE_FAST               1 (t)

 10           6 SETUP_LOOP              24 (to 33)
              9 LOAD_GLOBAL              1 (xrange)
             12 LOAD_FAST                0 (n)
             15 CALL_FUNCTION            1
             18 GET_ITER            
        >>   19 FOR_ITER                10 (to 32)
             22 STORE_FAST               2 (_)

 11          25 LOAD_FAST                1 (t)
             28 POP_TOP             
             29 JUMP_ABSOLUTE           19
        >>   32 POP_BLOCK           
        >>   33 LOAD_CONST               0 (None)
             36 RETURN_VALUE        


この LOAD_FAST が、その名前の通りに高速な名前解決を行う。関数のローカル変数は関数を実行する際の frameobject に配列の形式で保持されていて、ローカル変数の名前と配列のインデックスとの対応はバイトコード生成時に解決される。上のバイトコードには LOAD_FAST が二箇所でてくるが、LOAD_FAST の後にある 0 あるいは 1 が LOAD_FAST のオペランドで配列のインデックスを意味している。

ローカル変数 t については、3 STORE FAST 1 で、
```C
frameobject->f_localsplus[1] = typeオブジェクトのアドレス;
```
とされたあと、25 LOAD_FAST 1 で、
```C
*スタックの先頭 = frameobject->f_localsplus[1];
```
として名前の解決を行うため高速なのである。

### スペシャルメソッド検索の特異性

Python のスペシャルメソッド（\_\_xxx\_\_ の形式のメソッド）の多くは、加減乗除やシーケンスのスライスなど Python の演算子と強く結びついている。例えば、a * b に対して本質的に a.\_\_mul\_\_(b) が実行される。ただし、演算子としての記述から特殊メソッドを呼び出す場合と明示的に特殊メソッドを呼び出す場合とでは処理時間が異なる。加算を例に測定してみる。

In [13]:
def t1(n):
    for _ in xrange(n):
        n * 2
def t2(n):
    for _ in xrange(n):
        n.__mul__(2)
n = 1000000
for t in [t1, t2]:
    %timeit t(n)

10 loops, best of 3: 41.1 ms per loop
10 loops, best of 3: 140 ms per loop


どちらも最終的に \_\_mul\_\_ メソッドが呼び出されるが処理時間は圧倒的に演算子の方が早い。まずは、それぞれの関数のバイトコードを見る。

In [14]:
dis(t1)

  2           0 SETUP_LOOP              28 (to 31)
              3 LOAD_GLOBAL              0 (xrange)
              6 LOAD_FAST                0 (n)
              9 CALL_FUNCTION            1
             12 GET_ITER            
        >>   13 FOR_ITER                14 (to 30)
             16 STORE_FAST               1 (_)

  3          19 LOAD_FAST                0 (n)
             22 LOAD_CONST               1 (2)
             25 BINARY_MULTIPLY     
             26 POP_TOP             
             27 JUMP_ABSOLUTE           13
        >>   30 POP_BLOCK           
        >>   31 LOAD_CONST               0 (None)
             34 RETURN_VALUE        


In [15]:
dis(t2)

  5           0 SETUP_LOOP              33 (to 36)
              3 LOAD_GLOBAL              0 (xrange)
              6 LOAD_FAST                0 (n)
              9 CALL_FUNCTION            1
             12 GET_ITER            
        >>   13 FOR_ITER                19 (to 35)
             16 STORE_FAST               1 (_)

  6          19 LOAD_FAST                0 (n)
             22 LOAD_ATTR                1 (__mul__)
             25 LOAD_CONST               1 (2)
             28 CALL_FUNCTION            1
             31 POP_TOP             
             32 JUMP_ABSOLUTE           13
        >>   35 POP_BLOCK           
        >>   36 LOAD_CONST               0 (None)
             39 RETURN_VALUE        


t2 の方はこれまでにも出てきた LOAD_ATTR + CALL_FUNCTION であるが、t1 の方は BINARY_MULTIPLY になっている。LOAD_ATTR では検索する名前がスペシャルメソッドの `__add__` であっても通常の属性探索（n の指すインスタンス辞書、n のクラスの継承ツリー）が行なわれる。

一方、BINARY_MULTIPLY では n の指すインスタンス辞書を見ることも継承ツリーによる属性探索もなく PyNumber_Multiply を呼びだす。この関数では n の指すインスタンスが数値演算を実装していれば
```
(nの指すインスタンス)->ob_type->tp_as_number->nb_multiply(...)
```
を呼び出すため処理時間が短くなる。

#### 【参考】スペシャルメソッドの暗黙の使用で継承ツリーをたどる必要のない理由
スペシャルメソッドはクラスが C実装による組み込み型であるか、Pythonスクリプトで定義されたかによらず、必ず PyTypeObject (これはクラスのCによる内部実装)から辿れる構造体メンバー(関数ポインタ)として有効な関数のアドレスが設定されるようになっている。Python内部ではこの関数ポインタを保持するメンバーをスロット(slot)と呼んでいる。

クラスの定義時に親クラスから継承されるスロットは全てコピーされる。また、クラス生成後に、親クラスのクラス属性が変更された場合に、スロットが変更されていればそれを子孫のクラスに再反映するようになっている。このメカニズムによって、スペシャルメソッドの(暗黙の呼び出しによる)継承ツリーの検索を排除している。（ということは、クラス属性の変更はインスタンス属性の変更よりも負荷が大きいことを意味する）

### スライスとmemoryview

最後に空間パフォーマンスへの影響が大きい、シーケンスオブジェクトのスライス操作について述べる。文字列やリストのスライス操作は Python ではごく一般的に使用される。良く知られているように、これら代表的な二つのオブジェクトのスライス操作では、新しいオブジェクト(文字列やリスト)が再生されるが、その意味は文字列であれば byte 列、list であればオブジェクトを格納する配列が新規に作成されるということである。これは、一つのオブジェクトから大量のスライス演算をする場合に予想外の問題を引き起こす可能性を示唆する。

スライス操作で背景となるオブジェクトのバイト列を共有する方法として古くは buffer 関数が、Python 2.7 からは memoryview という機能が導入された。まずは、巨大なシーケンスから先頭を一つずつずらしたシーケンスを生成するコードで、単純なスライス演算と memoryview を介した場合の処理時間を比較する。

In [16]:
N = 100000
S = '*' * N

def t1(s):
    m = memoryview(s)
    for i in xrange(N):
        m = m[1:]
def t2(s):
    for i in xrange(N):
        s = s[1:]
        
for t in [t1, t2]:
    %timeit t(S)

100 loops, best of 3: 14 ms per loop
1 loop, best of 3: 322 ms per loop


この違いについてはあまり述べることはない。t1 の memoryview は s の指す文字列オブジェクトの土台となるバイト列(これはもはや Python上のオブジェクトではない)への参照を保持する memoryview オブジェクトを生成する。memoryview オブジェクトのスライス演算では、あくまで土台となるバイト列の参照位置(インデックス)を調整した新しい memoryview オブジェクトを戻すだけなので、文字列のスライス演算に対して非常に軽い処理となっている。これがそのまま処理時間として反映されている。

memoryview はネットワークソケットへの書き込みなどのように、バッファの一部分しか書き込めない可能性のある処理に向いている。ただし、上の例のような 1 要素ずつスライスをずらしていくような事は稀であるので、実際の性能のアドバンテージは僅かかもしれない。

#### 今ひとつな memoryview

効率を上げる方法として memoryview を紹介したが、もともと memoryview を試したきっかけとなった問題について、結果的に解決策とはならなかった事例を紹介する。それは、接尾辞配列(Suffix Array)の構築である。Suffix Array の構築はより効率の良いアルゴリズムが存在するが、その定義に基づく単純な方法が以下の関数である。

In [17]:
def make_sa(s):
    sa = [s[i:] for i in xrange(len(s))]
    return sorted(sa)

このように手軽に書けるのが Python の魅力であるが、この関数にわずか 100KB のデータを与えるだけで 5GB を越えるメモリが使用されてしまう。1MBのデータでは 512GB にもなり実用できない。そこで、memoryview で実装してみた。

In [18]:
def make_sa2(s):
    m = memoryview(s)
    sa = [m[i:] for i in xrange(len(s))]
    return sorted(sa)

一見よさそうだが、もどされたデータは並び順が正しくなかった。

In [19]:
data = 'Cython improve'
print make_sa(data)
print [m.tobytes() for m in make_sa2(data)]

[' improve', 'Cython improve', 'e', 'hon improve', 'improve', 'mprove', 'n improve', 'on improve', 'ove', 'prove', 'rove', 'thon improve', 've', 'ython improve']
['Cython improve', 'ython improve', 'thon improve', 'hon improve', 'on improve', 'n improve', ' improve', 'improve', 'mprove', 'prove', 'rove', 'ove', 've', 'e']


これは、memoryview の比較が土台となるオブジェクトの比較メソッドを使用しないことによる。そこで、sorted に比較のキーを取り出す関数を与えるか、比較の関数を与える。

In [20]:
def make_sa3(s):
    m = memoryview(s)
    sa = [m[i:] for i in xrange(len(s))]
    return sorted(sa, key=lambda m:m.tobytes())

In [21]:
data = 'Cython improve'
print make_sa(data)
print [m.tobytes() for m in make_sa3(data)]

[' improve', 'Cython improve', 'e', 'hon improve', 'improve', 'mprove', 'n improve', 'on improve', 'ove', 'prove', 'rove', 'thon improve', 've', 'ython improve']
[' improve', 'Cython improve', 'e', 'hon improve', 'improve', 'mprove', 'n improve', 'on improve', 'ove', 'prove', 'rove', 'thon improve', 've', 'ython improve']


こうすることで正しく動作するようにはなったが、比較の都度、memoryobject に対する tobytes メソッドの呼び出しが発生し何度も文字列オブジェクトのスライスが生成されることにより処理時間が妥当でなくなった。

この問題に対して Python スクリプト上の工夫で解決できる方法が思いつかない。

## Python の壁を越える

### Python 上の工夫では限界がある

先に、Pythonスクリプトの記述を改善することでより効率的になる例を検証してきたが、そのような方法では限界もある。
- 属性のルックアップコスト

  メソッドオブジェクトをキャッシュする、演算子を適切に使用するなどで、ある程度までは属性のルックアップコストを抑えることができるが、本質的に動的型付けという性格上、属性のルックアップにはほとんど常に動的ディスパッチを伴い、これを回避することはできない。


- 基本演算での関数呼び出し

  数値型の加減乗除などの極基本的な計算処理に対して PyNumber_xxx のような関数呼び出しが発生する。また、基本的な数値型 C言語の long/double などで済む演算であっても、二項演算の実行のたびに、オペランドの Pythonオブジェクトから Cのlong/double への変換、演算結果の Cのlong/double から Pythonオブジェクトへの変換が発生する。

もはや、このレベルを改善しようとすると、C/C++言語のような強い型付けの言語に頼るしかなくなる。

### C/C++の力を借りる

Python で C/C++ の力を借りるとすれば、C/C++で書かれたコードを Python に取り込む必要がある。これにはいくつかの方法がある。

#### 既存のパッケージ（拡張モジュール）を使う

真っ先にすべきは、Python に即 import 可能なパッケージ(拡張モジュール群、ライブラリ)を探すべきである。

- 標準ライブラリを活用

  collections, array, bisect など標準ライブラリには多くの拡張モジュールが備わっている。例えば、FIFO を安直に list で 実装していないだろうか？ collections.deque は list よりも効率的に FIFO を実現できる。
  

- 著名なパッケージを利用

  Python は特に科学技術計算系で豊富な実績あるパッケージが存在する。行列や数値演算でほぼ必須の numpy, scipy、統計処理の panda など、目的にあうパッケージを検索することは無駄にはならない。

#### 高速な C/C++ のライブラリを使う

目的にあった Python のパッケージがなくとも、既存の共有ライブラリやC/C++のソースから共有ライブラリを作成可能であれば、その共有ライブラリを Python から使用する方法がある。

- SWIG でラップする拡張ライブラリを作成 (半自動)

  SWIG (Simplified Wrapper and Interface Generator) により、共有ライブラリ用のヘッダーファイルから Python の拡張モジュールを生成することができる。


- 標準ライブラリの ctypes で C/C++ 関数を呼び出す (手軽)

  目的の共有ライブラリから数個の関数呼び出しが必要な程度であれば、標準ライブラリの ctypes を使用することもできる。ただし、SWIG と違い ctypes の場合は、共有ライブラリ側の関数のAPI情報(引数の型や戻り値)は手動で定義する必要があり、API呼び出しのオーバヘッドがあるので呼び出す関数内の処理の少なければ効果は低くなる。

#### Pythonのコードを C/C++ に移行する

先のどれにも該当しない場合は、Python で書かれたコードを C/C++ に移行するしかない。

- 高速化したい部分を C/C++ に移行する

  高速化したいコードが数値計算のみ含む単純なものであれば良いが、Pythonの辞書やリスト・タプルなどの処理と混在している場合は、その部分を適切に変更する必要がある。辞書やリストが計算結果として Python 側に戻すものであれば、Python API の PyDict_xxx, PyList_xxx を使用すべきであるし、Python側に出ないローカルなものであれば、C++の map や vector に置き返るなどの対応が必要である (Cの場合は自前で実装)。


- Pythonから使用するためのブリッジング

  C/C++ 化したコードを Python で利用するためのブリッジも必要となる。これは、はなから高速化コードを含む拡張モジュールとして作成するか、あるいは高速化コードを通常の共有ライブラリとして生成し、そこに前節のように SWIG / ctypes を使用する方法がある。

## Cython で越える

### Cython とは

Cython は Python に C風の型指定やいくつかの構文を追加した Python の上位言語である。Cython で記述されたコードは cython コマンドによりC言語に変換される。この変換コードには、Python の拡張モジュールのためのコードが含まれており、単純なコンパイルとリンクだけで拡張モジュールを生成できる。以下のような特徴をもつ。

- 静的型付け

  Cython の最大の特徴は、Python の変数に型を追加することである。型を追加することで静的解析によりコードを最適化することがねらいである。例えば、C言語の int/long/float 指定された型同士の算術演算は Python API (PyNumber_xxx) をバイパスできるため処理速度が大幅に向上する。
  

- C構造体の定義

  既存の型指定だけでなく、C言語の struct 同様に構造体を定義できる。これは、高速化だけでなく C/C++ とのバイナリデータの交換も容易であることを意味する。
  
  
- ポインタ演算

  C言語同様にポインタ演算 (C系データのアドレス取得、間接参照、オフセット加算など) が可能である。
  
  
- C-API形式の関数(メソッド)の定義・呼び出しが可能

  Pythonで定義される関数は PyObject * を引数や返却値とする。Cython では特殊なキーワードを指定することで、引数の型や返却値の型を C言語の任意の型にすることができる。そのような関数は Python スクリプト側からは呼び出せないが、Cython 内において高速に呼び出しができる。

### Cython で出来ること

上のような特徴を持つ Cython は Pythonのコードを C/C++ に移行するのに優れたツールとなる。

#### 高速な C/C++ のライブラリを使う

- APIに関して手作業で型などを記述する必要性はあるものの、C言語からの宣言の移行は簡単であり、少ない手間で拡張ライブラリを容易に作成できる。

- Cython は Python に特化しているため、SWIGよりも最適化されたコードが生成される(らしい)。

#### Pythonのコードを C/C++ に移行する

- Pythonコードを最大限に再利用できる (構文をほぼ維持)

### Cython の使用方法

Cython を使用する方法は以下のように単純である。

1. Cython 文法で .pyx ファイルを書く。

2. cython コマンドで .pyx から .c を生成。

3. .c から共有ライブラリ(.so/.dll)を生成。これで拡張モジュールとなる!!

4. できた拡張モジュールを普通に import するだけ。

以下、もう少し具体的に説明する。

#### Cython 文法で .pyx ファイルを書く

既存の Python コードを Cython 化する想定のもと、以下のようなコードがあったとする。

sample.py:
```python
def put_rbuf(self, addr, size, c_obj):

    ubuf = c_addressof(c_obj)
    rbuf = self._top + (addr % self._size)
    rrest = self._end - rbuf

    while size > 0:
        cpsize = size if size < rrest else rrest
        c_memmove(rbuf, ubuf, cpsize)
        ubuf += cpsize
        size -= cpsize
        rbuf = self._top
        rrest = self._size
```

これを、次のような Cython のコード(samply.pyx)に書き換える。

sample.pyx:
```cython
cdef void put_rbuf(self, c_uint32 addr, c_uint32 size, void *c_obj):
    cdef:
        char *ubuf = <char *>c_obj
        char *rbuf = self._top + (addr % self._size)
        c_uint32 rrest = self._end - rbuf
        c_uint32 cpsize
    while size > 0:
        cpsize = size if size < rrest else rrest
        memcpy(rbuf, ubuf, cpsize)
        ubuf += cpsize
        size -= cpsize
        rbuf = self._top
        rrest = self._size
```
ここで両者を見比べれば、いくつかのキーワード `cdef` や型宣言 `c_uint32`、C風型変換 `<char *>` が付加されているが、その他の記述はもとの Python のままであることがわかる。

#### cython コマンドで .pyx から .c を生成

```
$ cython sample.pyx
```
- これで sample.c が生成される。

- `-a` オプションを指定すると、cython のソースと生成された Cコードを対比を確認できる html ファイルが生成される。

#### .c から拡張モジュールを生成

あとは、拡張モジュールの生成に必要なオプションを指定して共有ライブラリを作成するだけである。

```
$ cc -c -fPIC $(python-config --cflags) sample.c
$ cc -shared -o sample.so $(python-config --ldflags) sample.o
```

生成された sample.so は直ちに import 可能な拡張モジュールとなっている。

#### \[おまけ\] Pythonコードをそのまま使用

Pythonコードはそのまま Cython のコードでもあるため、cython コマンドで拡張モジュールを生成することができる。この場合、静的型指定による高速化の恩恵は得られないが、Python がバイトコードを解釈する処理はバイパスできる。生成されたコードは、ほぼ Python がバイトコードを解釈実行する事で呼ばれる Python API に相当するため、生成されたコードを見ることは Python がどのように動作しているかを知る参考情報となる。

### Cython で早くなる理由

Cython化することで早くなる理由を、cythonコマンドで生成されるC言語のコードから検証する。以降では、Jupyter notebook のセル内で cython コードを使えるように notebook の拡張機能をロードする。

In [22]:
%load_ext cython

#### 数値演算を比べてみる
Cython でもっとも期待されるであろう数値演算について、型指定の有無による違いを見る。

In [23]:
%%cython -a
def f(a, b, long c, long d):
    a + b
    c + d

  上のコードの +2 や +3 をクリックする cython で生成されたCのコードが表示される。

2行目の `a + b` は、`a` も `b` も**型指定されていないため、PyNumber_Add**という Python API が呼ばれている。PyNumber_Add は数値オブジェクトに対して呼ばれる訳ではなく、二項加算演算の式に対して呼ばれるものである。

一方 3行目の `c + d` は `c` と `d` が仮引数リスト内において `long` の**型指定があるため、そのまま Cの加算コード**が生成されている。

#### メソッド呼び出しを比べてみる

次にメソッド呼び出しのコードを比べてみる。これは前の例ほど単純ではないので、まず検証対象とする Cython のコードについて先に解説しておく。

```cython
cdef class C(object):
    cdef void c_method(self, int a, int b, int c):
        pass
    def p_method(self, a, b, int c):
        pass

def f(a, C c):
    a.method(1,2,3)
    c.c_method(1,2,3)
```

- `cdef class` は拡張モジュールとしてクラスを定義する。これは [Python インタプリタの拡張と埋め込み](http://docs.python.jp/2/extending/index.html)で説明されているように、Cレベルでクラスを定義することを意味する。このようなクラスインスタンスのインスタンス属性は構造体のフィールドとしてあらかじめ準備されたり、Pythonのインタープリタで動的に作成されるクラスとは異なる。

- c_method には `cdef` 指定(と戻りと引数の型指定)があるが、これは Cython上で C関数と呼ばれるものである。しかし、Cython 上の `cdef` 指定のない `def` による関数 p_method も C言語の関数に落とされるため若干混乱がある。`cdef` 指定のある関数は型宣言のままの C関数となるが、`cdef` 指定がない関数は Pythonスクリプト側からの呼び出しが可能な関数であり、その規約に従って `self`引数のあとは、`PyObject *args`, `PyObject *kwargs` になっている。

- 関数 `f` 内での二つの呼び出しは微妙な違いがある。最初の `a.method` は `a` が型指定を持っていない。一方、`c.c_method` は `c` がこの Cython コードで定義される `cdef`指定クラスの `C` のオブジェクトであることがわかっている。この違いが生成されるコードにどう影響するかを検証するのが、このサンプルの目的である。

In [24]:
%%cython -a
cdef class C(object):
    cdef void c_method(self, int a, int b, int c):
        pass
    def p_method(self, a, b, int c):
        pass

def f(a, C c):
    a.method(1,2,3)
    c.c_method(1,2,3)

#### cdef関数(C関数)と def関数(Python関数) との違い

最初に `cdef`指定のある関数(C関数)と、指定のない関数(Python関数)との違いを見る。

2行目の `c_method` の宣言は概ね以下のようになっている。
```C
static void __pyx_省略_1C_c_method(struct __pyx_省略_C *__pyx_v_self,
                                   int __pyx_v_a, int __pyx_v_b, int __pyx_v_c)
```
これはほぼ `cdef` 宣言そのままである。

一方 4行目の `p_method` の宣言部分から生成されるコードは非常に大きい。似たような名前の関数が2つ生成されているが、エッセンスを抜きだすと以下のようになる。
```C
static PyObject *__pyx_省略_1C_1p_method(PyObject *__pyx_v_self,
                                         PyObject *__pyx_args,
                                         PyObject *__pyx_kwds)
{
    __pyx_args, __pyx_kwds から引数オブジェクトを確定し values[] へ;
    __pyx_v_a = values[0];
    __pyx_v_b = values[1];
    __pyx_v_c = __Pyx_PyInt_As_int(values[2]);
    __pyx_r = __pyx_省略_1C_p_method(((struct __pyx_省略_C *)__pyx_v_self),
                                     __pyx_v_a, __pyx_v_b, __pyx_v_c);
    return __pyx_r;
}
static PyObject *__pyx_省略_1C_p_method(struct __pyx_省略_C *__pyx_v_self,
                                        PyObject *__pyx_v_a, PyObject *__pyx_v_b, int __pyx_v_c)
{
    __pyx_r = Py_None; __Pyx_INCREF(Py_None);
    __Pyx_XGIVEREF(__pyx_r);
    return __pyx_r;
}
```
- 最初にあげた関数は、Pythonのスクリプトから呼ばれる関数である
- 二番目の関数は Cython 上で呼び出す場合の関数であり Cython の `p_method` 本体に記述した処理はこちらの関数の方で実装される。
- 最初の関数は Python から規約通りに呼ばれるため、引数リスト(tuple形式)やキーワード引数(dict形式)の解釈と引数の個数のチェックが行われる。

Python関数は cdef指定のC関数よりも多くの仕事をしていることがわかる。

##### a.method の呼び出し
8行目の `a.a_method` であるが、生成されるコードの順番により多少わかりにくくなっているので、エラー処理や参照カウント処理を取り除いたエッセンスを整理すると、以下のようなコードになる。
```C
__pyx_tuple_ = PyTuple_Pack(3, __pyx_int_1, __pyx_int_2, __pyx_int_3);
__pyx_t_1 = __Pyx_PyObject_GetAttrStr(__pyx_v_a, __pyx_n_s_method);
__pyx_t_2 = __Pyx_PyObject_Call(__pyx_t_1, __pyx_tuple_, NULL);
```
- Python関数の呼び出し規則に従って、引数が tuple にまとめられている。なお、`__pyx_int_1`, `__pyx_int_2`, `__pyx_int_3` はすべて `PyObject *` であり Python の整数オブジェクトの参照である。
- `a` の指すオブジェクトに対して文字列`"method"` の属性探索が行われる。
- 最後に見つかった属性に対して汎用の呼び出し機構である `PyObjcet_Call` で呼び出しを行う。この関数の最後の引数はキーワード引数用の辞書オブジェクトのアドレスを渡すところで、ここでは固定引数のみなので NULL が渡されている。

##### c.c_method の呼び出し
一方で、9行目の `c.c_method` からは、
```C
((struct __pyx_省略_C *)__pyx_v_c->__pyx_vtab)->c_method(__pyx_v_c, 1, 2, 3)
```
というコードが生成される。先程のコードと対比すると以下の違いがあり、これらが早さの秘密である。
- 構造体メンバーの間接参照のみで C言語の関数ポインタを通して `c_method` を呼び出している。
- 引数も `PyObject *` へ変換することなく C言語の型のまま渡されている。

## Cythonの適用例

最後に実際に使用していた Python のコードを Cython 化した適用例を示して締めくくる。これは、メモリマップ(mmap)した固定のエリアに巡回的にログを書き込む機能を cython 化した例である。

- C言語用としてライブラリは作成されていた。

- そのライブラリへの依存性を排除するため pure python で実装していた。

- やはり遅いため cython で実装しなおした。

### 実測値

1MBytes の領域に 40 bytes のデータを 1000000 回書き込む (約 38MBytes)。

CPU: Intel(R) Xeon(R) X5680 3.33GHz

実装方法               | 経過時間(sec) | 対 pure Python
-----------------------|---------------|----------------
pure Python            | <center>41.194<center/> | 
ctypes によるブリッジ  | <center>1.729<center/> | <center>23.8<center/>
Cython による再実装    | <center>0.214<center/> | <center>192.5<center/>
Cython によるブリッジ  | <center>0.208<center/> | <center>198.0<center/>
C によるオリジナル     | <center>0.134<center/>  | <center>316.9<center/>

### 考察

- Cython を使用することで外部ライブラリへの依存を排除したうえでパフォーマンスが向上した。
- Cython で外部ライブラリブリッジを作成することは容易であり、パフォーマンス上も満足がいく。
- ctypes と cython によるブリッジとは同じCの関数を呼び出している。
  汎用機構である ctypes の C関数呼び出しのオーバヘッドの大きさがわかる。  
- やはり、C言語は爆速だった。

### Cython は完璧か？

- 数値計算、バイト列に対するポインタ演算などを多用するコードは Cython での高速化が望める。
- Pythonのオブジェクトを多用するコードは、Python内部表現とCの型との boxing/unboxing の頻度が増えることを考慮する必要がある。
- Cythonのもつ型変換がトラップとなる場合がある：
  - `char *` の引数渡しの際に、`PyString_Object` が生成され、そのオブジェクト中の文字バッファのアドレスが代わりに渡される。

### まとめ

- Python はその動的特性ゆえに Python コード自体では回避できないパフォーマンスの問題がある。
- パフォーマンスの問題で Python をためらっているなら Cython 化という方法がある。
- Cython 化は C に書き直すよりも遥かに容易であるため、試してみて損はない。